In [1]:
using DynamicHMCModels

ProjDir = rel_path_d("..", "scripts", "12")

df = CSV.read(rel_path( "..", "data",  "Kline.csv"), delim=';');
size(df) # Should be 10x5

(10, 5)

New col logpop, set log() for population data

In [2]:
df[:society] = 1:10;
df[:logpop] = map((x) -> log(x), df[:population]);
#df[:total_tools] = convert(Vector{Int64}, df[:total_tools])
first(df[[:total_tools, :logpop, :society]], 5)

,total_tools,logpop,society
,Int64⍰,Float64,Int64
1,13,7.00307,1
2,22,7.31322,2
3,24,8.18869,3
4,43,8.47449,4
5,33,8.90924,5


Define problem data structure

In [3]:
struct m_12_06d{TY <: AbstractVector, TX <: AbstractMatrix,
  TS <: AbstractVector}
    "Observations (total_tools)."
    y::TY
    "Covariates (logpop)"
    X::TX
    "Society"
    S::TS
    "Number of observations (10)"
    N::Int
    "Number of societies (also 10)"
    N_societies::Int
end;

Make the type callable with the parameters *as a single argument*.

In [4]:
function (problem::m_12_06d)(θ)
    @unpack y, X, S, N, N_societies = problem   # extract the data
    @unpack β, α, s = trans(θ)  # β : a, bp, α : a_society, s
    σ = s[1]^2
    ll = 0.0
    ll += logpdf(Cauchy(0, 1), σ) # sigma
    ll += sum(logpdf.(Normal(0, σ), α)) # α[1:10]
    ll += logpdf.(Normal(0, 10), β[1]) # a
    ll += logpdf.(Normal(0, 1), β[2]) # bp
    ll += sum(
      [loglikelihood(Poisson(exp(α[S[i]] + dot(X[i, :], β))), [y[i]]) for i in 1:N]
    )
end

Instantiate the model with data and inits.

In [5]:
N = size(df, 1)
N_societies = length(unique(df[:society]))
X = hcat(ones(Int64, N), df[:logpop]);
S = df[:society];
y = df[:total_tools];
γ = (β = [1.0, 0.25], α = rand(Normal(0, 1), N_societies), s = [0.2]);
p = m_12_06d(y, X, S, N, N_societies);

Function convert from a single vector of parms to parks NamedTuple

In [6]:
trans = as((β = as(Array, 2), α = as(Array, 10), s = as(Array, 1)));

Define input parameter vector

In [7]:
θ = inverse(trans, γ);
p(θ)

-4414.823391947998

Maximum_a_posterior

In [8]:
using Optim

x0 = θ;
lower = vcat([0.0, 0.0], -3ones(10), [0.0]);
upper = vcat([2.0, 1.0], 3ones(10), [5.0]);
ll(x) = -p(x);

inner_optimizer = GradientDescent()

res = optimize(ll, lower, upper, x0, Fminbox(inner_optimizer));
res

Results of Optimization Algorithm
 * Algorithm: Fminbox with Gradient Descent
 * Starting Point: [1.0,0.25, ...]
 * Minimizer: [1.0733073546274228,0.2664261997004786, ...]
 * Minimum: -1.353734e+02
 * Iterations: 1000
 * Convergence: false
   * |x - x'| ≤ 0.0e+00: false 
     |x - x'| = 1.74e-10 
   * |f(x) - f(x')| ≤ 0.0e+00 |f(x)|: false
     |f(x) - f(x')| = -3.32e-07 |f(x)|
   * |g(x)| ≤ 1.0e-08: false 
     |g(x)| = 2.58e+05 
   * Stopped by an increasing objective: false
   * Reached Maximum Number of Iterations: false
 * Objective Calls: 128463408
 * Gradient Calls: 128463408

Minimum gives MAP estimate:

In [9]:
Optim.minimizer(res)

13-element Array{Float64,1}:
  1.0733073546274228    
  0.2664261997004786    
 -3.2660289601103933e-14
  4.1924195702002746e-14
  2.2728160135594888e-14
  1.95148749900423e-13  
  7.476231543904027e-14 
 -6.806177676366784e-14 
  1.327539834538126e-13 
 -5.274583408550584e-15 
  2.5060784578532843e-13
  2.2670091800172434e-12
  7.756979159719898e-5  

Write a function to return properly dimensioned transformation.

In [10]:
problem_transformation(p::m_12_06d) =
  as( Vector, length(θ) )
# Wrap the problem with a transformation, then use ForwardDiff for the gradient.
P = TransformedLogDensity(problem_transformation(p), p)
∇P = LogDensityRejectErrors(ADgradient(:ForwardDiff, P));
#∇P = ADgradient(:ForwardDiff, P);

Tune and sample.

In [11]:
chain, NUTS_tuned = NUTS_init_tune_mcmc(∇P, 1000);

MCMC, adapting ϵ (75 steps)
0.0052 s/step ...done
MCMC, adapting ϵ (25 steps)
0.0054 s/step ...done
MCMC, adapting ϵ (50 steps)
0.0082 s/step ...done
MCMC, adapting ϵ (100 steps)
0.0034 s/step ...done
MCMC, adapting ϵ (200 steps)
0.0015 s/step ...done
MCMC, adapting ϵ (400 steps)
0.00086 s/step ...done
MCMC, adapting ϵ (50 steps)
0.00082 s/step ...done
MCMC (1000 steps)
0.00073 s/step ...done


We use the transformation to obtain the posterior from the chain.

In [12]:
posterior = TransformVariables.transform.(Ref(problem_transformation(p)), get_position.(chain));
posterior[1:5]

5-element Array{Array{Float64,1},1}:
 [0.15652723786107908, 0.3601805261347912, -0.18488983165457992, 0.3258766520363213, 0.12109368209728455, 0.3957213633931597, 0.06908418219624492, -0.0825578554354227, 0.182218031601832, -0.2672074128593706, 0.2122598098962865, -0.37179831644622163, 0.5711393018331508]        
 [0.7848403165172436, 0.29376882282648664, 0.1387070875963687, -0.015130692963375658, 0.13676059668966284, 0.41790213828563527, 0.15498138062961864, -0.24409944161917557, 0.1522430991559138, -0.3804634562687419, 0.22412175815796342, -0.16924379075222928, 0.6324215722527609]   
 [-0.5424760753177695, 0.45830575048688355, -0.14502294923342024, 0.31090200333191365, -0.09526305790329209, 0.40285642613827133, 0.02114204736220831, -0.547320977605547, -0.16866119950733666, -0.27775270415365183, 0.0515139724801193, -0.9662865673598149, 0.6480675997464947]  
 [0.8266534682273761, 0.3008119919984784, -0.11441491697472866, 0.17396458033716117, -0.12470854863680525, 0.5221566277108983, 0.

Extract the parameter posterior means.

In [13]:
posterior_β = mean(trans(posterior[i]).β for i in 1:length(posterior))
posterior_α = mean(trans(posterior[i]).α for i in 1:length(posterior))
posterior_σ = mean(trans(posterior[i]).s for i in 1:length(posterior))[1]^2

0.2892828907567338

Effective sample sizes (of untransformed draws)

In [14]:
ess = mapslices(effective_sample_size, get_position_matrix(chain); dims = 1)
ess

1×13 Array{Float64,2}:
 1000.0  1000.0  990.475  1000.0  1000.0  …  1000.0  1000.0  1000.0  463.613

NUTS-specific statistics

In [15]:
NUTS_statistics(chain)

Hamiltonian Monte Carlo sample of length 1000
  acceptance rate mean: 0.9, min/25%/median/75%/max: 0.05 0.86 0.95 0.99 1.0
  termination: AdjacentTurn => 9% DoubledTurn => 91%
  depth: 2 => 0% 3 => 16% 4 => 84%


CmdStan result

In [16]:
m_12_6_result = "
Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
                            Mean                SD               Naive SE             MCSE            ESS
            a          1.076167468  0.7704872560 0.01218247319 0.0210530022 1000.000000
           bp         0.263056273  0.0823415805 0.00130193470 0.0022645077 1000.000000
  a_society.1   -0.191723568  0.2421382537 0.00382854195 0.0060563054 1000.000000
  a_society.2    0.054569029  0.2278506876 0.00360263570 0.0051693148 1000.000000
  a_society.3   -0.035935050  0.1926364647 0.00304584994 0.0039948433 1000.000000
  a_society.4    0.334355037  0.1929971201 0.00305155241 0.0063871707  913.029080
  a_society.5    0.049747513  0.1801287716 0.00284808595 0.0043631095 1000.000000
  a_society.6   -0.311903245  0.2096126337 0.00331426674 0.0053000536 1000.000000
  a_society.7    0.148637507  0.1744680594 0.00275858223 0.0047660246 1000.000000
  a_society.8   -0.164567976  0.1821341074 0.00287979309 0.0034297298 1000.000000
  a_society.9    0.277066965  0.1758237250 0.00278001719 0.0055844175  991.286501
 a_society.10   -0.094149204  0.2846206232 0.00450024719 0.0080735022 1000.000000
sigma_society    0.310352849  0.1374834682 0.00217380450 0.0057325226  575.187461
";

Show means

In [17]:
[posterior_β, posterior_α, posterior_σ]

3-element Array{Any,1}:
  [1.1349726727521237, 0.25720405990787437]                                                                                                                                                                            
  [-0.20703238521134099, 0.03936912846769978, -0.04210206871654808, 0.32395510769469954, 0.04449599315078086, -0.31314475526530405, 0.14602929458714461, -0.17065437666161976, 0.2758033713874104, -0.0834128873400268]
 0.2892828907567338                                                                                                                                                                                                    

End of m12.6d1.jl

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*